In [8]:
import gym
import torch
from torch import nn

In [3]:
environment = gym.make('CartPole-v1')

In [4]:
state = environment.reset()

In [5]:
state

array([-0.04252762, -0.03842452, -0.01699254, -0.03133183], dtype=float32)

In [6]:
environment.action_space

Discrete(2)

In [22]:
environment.step(0)

(array([-0.04796209, -0.42816475, -0.01250035,  0.5430156 ], dtype=float32),
 1.0,
 False,
 {})

In [9]:
class Network(nn.Module):
    def __init__(self, N_state, N_hidden):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(N_state, N_hidden),
            nn.ReLU(),
            nn.Linear(N_hidden, N_hidden),
            nn.ReLU(),
            nn.Linear(N_hidden, 1),
            nn.Sigmoid()
        )
        
        
    def forward(self, x):
        x = self.net(x)
        return x

In [17]:
net = Network(4, 4)

In [18]:
net(torch.from_numpy(state))

tensor([0.4979], grad_fn=<SigmoidBackward0>)

In [19]:
net.parameters()

<generator object Module.parameters at 0x1627b4b30>

In [20]:
net.state_dict()

OrderedDict([('net.0.weight',
              tensor([[-0.0160,  0.2757, -0.0294,  0.2686],
                      [-0.1165, -0.0253, -0.1185, -0.3917],
                      [ 0.4059, -0.1830, -0.1938, -0.0633],
                      [ 0.1620, -0.2424, -0.0303,  0.0828]])),
             ('net.0.bias', tensor([ 0.0467, -0.0472,  0.2009, -0.4035])),
             ('net.2.weight',
              tensor([[ 0.0873, -0.1049,  0.4551, -0.1541],
                      [ 0.3983,  0.3628, -0.4833,  0.4089],
                      [ 0.1993, -0.2436,  0.4484,  0.4685],
                      [ 0.0943, -0.1606,  0.0603,  0.2916]])),
             ('net.2.bias', tensor([0.3230, 0.1074, 0.2505, 0.1426])),
             ('net.4.weight', tensor([[-0.1777,  0.2864, -0.4699, -0.0959]])),
             ('net.4.bias', tensor([0.2352]))])

In [ ]:
# write fn to flatten and unflatten parameters and convert to np